This jupyter notebook contains the code to calculate distances between all pairs of cities. These distances are calculated in miles

In [23]:
import arcpy
import numpy as np

In [ ]:
## Path to ctu polygons, this will need to be replaced with a pull from the database at some point
ctu_polygons_path = r"C:\Users\aoheg\Desktop\School\GIS 5572\GIS5572\ctus_with_id.shp"

In [ ]:
# Make OID/UNIQUE_ID dictionary
codes_dict = {}
with arcpy.da.SearchCursor(ctu_polygons_path, ["FID", "UNIQUE_ID","POPULATION"]) as cursor:
    for row in cursor:
        codes_dict[row[0]] = row[1]

In [ ]:
## Convert to centroid points
arcpy.management.Delete("memory/ctu_points") # clear memory
arcpy.management.FeatureToPoint(ctu_polygons_path, "memory/ctu_points")

## Calculate all distances
arcpy.management.Delete("memory/ctu_distances") # clear memory
arcpy.analysis.GenerateNearTable("memory/ctu_points", "memory/ctu_points", "memory/ctu_distances", closest="ALL", distance_unit="Miles")

<Result 'memory\\ctu_distances'>

In [ ]:
## put into array
distances = np.zeros([2743, 2743])
with arcpy.da.SearchCursor("memory/ctu_distances", ["IN_FID","NEAR_FID","NEAR_DIST"]) as cursor:
    for row in cursor:
        distances[codes_dict[row[0]-1],codes_dict[row[1]-1]] = row[2] # for some reason, the -1 is necessary to line it up. I'm not sure why it is off by one
distances

array([[  0.        , 115.55560466,  98.42056273, ..., 219.99154627,
        211.05230116, 209.93731429],
       [115.55560466,   0.        ,  17.16908454, ..., 109.13584175,
         99.71444676,  97.87829874],
       [ 98.42056273,  17.16908454,   0.        , ..., 125.39285598,
        116.04433726, 114.36831882],
       ...,
       [219.99154627, 109.13584175, 125.39285598, ...,   0.        ,
          9.50844555,  12.22706283],
       [211.05230116,  99.71444676, 116.04433726, ...,   9.50844555,
          0.        ,   3.81635606],
       [209.93731429,  97.87829874, 114.36831882, ...,  12.22706283,
          3.81635606,   0.        ]])